In [1]:
import pandas as pd
import numpy as np
from navec import Navec
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from TextDataset import TextDataset
from lib import tokenizer, vocab, vectorize_batch
from models.RNN import RNN

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

N = 25000
batch_size = 128
vocab_length = 500002

In [3]:
navec = Navec.load("../models/navec_hudlit_v1_12B_500K_300d_100q.tar")

In [4]:
train_dataset, test_dataset  = TextDataset('../data/data.csv', N, 0.8, "head"), TextDataset('../data/data.csv', N, 0.2, "tail")
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2])

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, collate_fn=vectorize_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=vectorize_batch)

In [5]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

torch.Size([128, 50]) torch.Size([128])


In [6]:
rnn = RNN().to(device)

In [7]:
for layer in rnn.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print()

Layer : Embedding(500002, 300)
Parameters : 
torch.Size([500002, 300])

Layer : RNN(300, 20, batch_first=True)
Parameters : 
torch.Size([20, 300])
torch.Size([20, 20])
torch.Size([20])
torch.Size([20])

Layer : Linear(in_features=20, out_features=15, bias=True)
Parameters : 
torch.Size([15, 20])
torch.Size([15])



In [ ]:
loss_model = nn.CrossEntropyLoss()
opt = torch.optim.Adam(rnn.parameters(), lr=0.001)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,
                                                          mode='min',
                                                          factor=0.1,
                                                          patience=7,
                                                          threshold=1e-4,
                                                          threshold_mode='rel',
                                                          cooldown=3,
                                                          min_lr=0,
                                                          eps=1e-8,
                                                          )

: 

In [ ]:
EPOCHS = 5
train_loss, train_acc, val_loss, val_acc = [], [], [], []
train_precision, train_recall, train_f1 = [], [], []
val_precision, val_recall, val_f1 = [], [], []
list_ = []

for epoch in range(EPOCHS):
    rnn.train()
    running_train_loss = []
    all_train_preds, all_train_targets = [], []
    train_loop = tqdm(train_loader, leave=False)

    for x, targets in train_loop:
        x = x.to(device)
        targets = targets.to(device)

        pred = rnn(x)
        loss = loss_model(pred, targets)

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_train_loss.append(loss.item())

        preds = pred.argmax(dim=1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_targets.extend(targets.cpu().numpy())

        train_loop.set_description(
            f"Epoch {epoch + 1}/{EPOCHS};   loss={np.mean(running_train_loss):.4f}"
        )

    mean_train_loss = np.mean(running_train_loss)
    acc_train = np.mean(np.array(all_train_preds) == np.array(all_train_targets))
    prec_train = precision_score(all_train_targets, all_train_preds, average="weighted")
    rec_train = recall_score(all_train_targets, all_train_preds, average="weighted")
    f1_train = f1_score(all_train_targets, all_train_preds, average="weighted")

    train_loss.append(mean_train_loss)
    train_acc.append(acc_train)
    train_precision.append(prec_train)
    train_recall.append(rec_train)
    train_f1.append(f1_train)

    rnn.eval()
    with torch.no_grad():
        running_val_loss = []
        all_val_preds, all_val_targets = [], []
        for x, targets in val_loader:
            x = x.to(device)
            targets = targets.to(device)

            pred = rnn(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())

            preds = pred.argmax(dim=1)
            all_val_preds.extend(preds.cpu().numpy())
            all_val_targets.extend(targets.cpu().numpy())

        mean_val_loss = np.mean(running_val_loss)
        acc_val = np.mean(np.array(all_val_preds) == np.array(all_val_targets))
        prec_val = precision_score(all_val_targets, all_val_preds, average="weighted", zero_division=0)
        rec_val = recall_score(all_val_targets, all_val_preds, average="weighted", zero_division=0)
        f1_val = f1_score(all_val_targets, all_val_preds, average="weighted", zero_division=0)

        lr_scheduler.step(metrics=mean_val_loss)
        lr = lr_scheduler.get_last_lr()
        list_.append(lr)

        val_loss.append(mean_val_loss)
        val_acc.append(acc_val)
        val_precision.append(prec_val)
        val_recall.append(rec_val)
        val_f1.append(f1_val)

    print(
        f"Epoch {epoch+1}/{EPOCHS}; "
        f"train_loss={mean_train_loss:.4f}; train_acc={acc_train:.4f}; "
        f"train_prec={prec_train:.4f}; train_rec={rec_train:.4f}; train_f1={f1_train:.4f}; "
        f"val_loss={mean_val_loss:.4f}; val_acc={acc_val:.4f}; "
        f"val_prec={prec_val:.4f}; val_rec={rec_val:.4f}; val_f1={f1_val:.4f}"
    )

Epoch 1/5;   loss=2.9640:   1%|          | 1/125 [00:05<11:44,  5.68s/it]

In [ ]:
rnn.eval()
with torch.no_grad():
    running_test_loss = []
    all_test_preds, all_test_targets = [], []
    for x, targets in test_loader:
        x = x.to(device)
        targets = targets.to(device)

        pred = rnn(x)
        loss = loss_model(pred, targets)

        running_test_loss.append(loss.item())

        preds = pred.argmax(dim=1)
        all_test_preds.extend(preds.cpu().numpy())
        all_test_targets.extend(targets.cpu().numpy())

    mean_test_loss = np.mean(running_test_loss)
    acc_test = np.mean(np.array(all_test_preds) == np.array(all_test_targets))
    prec_test = precision_score(all_test_targets, all_test_preds, average="weighted")
    rec_test = recall_score(all_test_targets, all_test_preds, average="weighted")
    f1_test = f1_score(all_test_targets, all_test_preds, average="weighted")

print(
    f"test_loss={mean_test_loss:.4f}; test_acc={acc_test:.4f}; "
    f"test_prec={prec_test:.4f}; test_rec={rec_test:.4f}; test_f1={f1_test:.4f}"
)

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.legend(['train', 'val'])
plt.show()